In [34]:
from pymongo import MongoClient
import pandas as pd
import folium
import branca
from bson import ObjectId

In [2]:
def connectCollection(database, collection):
    client = MongoClient()
    db = client[database]
    coll = db[collection]
    return db, coll


In [3]:
db, coll = connectCollection('companies','design_new')

In [22]:
#funcion que nos dice lo que hay cerca de companies
def getNearby(lat,lng,comptype,radius=300):
    db, coll = connectCollection('companies',comptype)
    results = list(coll.find(
        {"coords":
         {"$near":
          {"$geometry":
           {
               "type":"Point",
               "coordinates":[lng,lat]
           },
           "$maxDistance":radius
          }
         }
        }
    ))
    total = len(results)
    #print("Total", total)
    return results if total > 0 else None

getNearby(37.4374154,-122.1722525,"party")

In [5]:
#ejemplo de una coordenada y una collection
#party=getNearby(37.4374154,-122.1722525,"party")

In [6]:
#print(party)

In [23]:
#aqui hago la query para extraer todas las empresas y voy haciendo un loop para que busque 
#cada uno de los venues por cada oficina
design_comp = list(coll.find({}))
venues=['party','schools','starbucks','vegan']
companies_nearby = []
for company in design_comp:
    long,lat = company['coords']['coordinates']
    companies_nearby.append({company["_id"]:{poi:getNearby(lat,long,poi) for poi in venues}})
companies_nearby

[{ObjectId('5dd6b0add27a078c8c61cb5f'): {'party': None,
   'schools': None,
   'starbucks': None,
   'vegan': None}},
 {ObjectId('5dd6b0add27a078c8c61cb60'): {'party': None,
   'schools': None,
   'starbucks': None,
   'vegan': None}},
 {ObjectId('5dd6b0add27a078c8c61cb61'): {'party': None,
   'schools': None,
   'starbucks': None,
   'vegan': None}},
 {ObjectId('5dd6b0add27a078c8c61cb62'): {'party': [{'_id': ObjectId('5dd4383535e2f878c2105b04'),
     'coords': {'coordinates': [-122.1639480591, 37.4450645447],
      'type': 'Point'},
     'country': 'US',
     'name': 'the-patio-palo-alto'}],
   'schools': None,
   'starbucks': [{'_id': ObjectId('5dd4326f35e2f878c2105880'),
     'coords': {'coordinates': [-122.1606364646, 37.44710776],
      'type': 'Point'},
     'country': 'US',
     'name': 'paris-baguette-palo-alto'},
    {'_id': ObjectId('5dd4326f35e2f878c2105858'),
     'coords': {'coordinates': [-122.160561, 37.446791], 'type': 'Point'},
     'country': 'US',
     'name': 'starb

In [19]:
len(companies_nearby)

37

In [9]:
#companies_nearby[0][ObjectId('5dd683b9d27a078c8c61cb39')]

In [24]:
#buscamos las oficinas que tenga los cuatro requerimientos 
final_companies=[]
for company in companies_nearby:
    #print(company)
    for c in company.values():
        if c['party'] is not None and c['schools'] is not None and c['vegan'] is not None and c['starbucks'] is not None:
            print('ok')
            final_companies.append(company)
            #companies_nearby.pop(companies_nearby.index(company))  

-----------
-----------
-----------
-----------
-----------
-----------
-----------
-----------
-----------
-----------
-----------
ok
-----------
-----------
-----------
-----------
-----------
-----------
-----------
-----------
-----------
-----------
-----------
-----------
-----------
-----------
-----------
-----------
-----------
-----------
-----------
-----------
-----------
-----------
-----------
-----------
-----------
-----------


In [48]:
selected=final_companies[0]

In [80]:
final_comp = list(coll.find({'_id':ObjectId('5dd6b0add27a078c8c61cb69')}))
final_comp[0]['coords']['coordinates']

[-122.1612868, 37.4440981]

In [57]:
#final_comp[ObjectId('5dd6b0add27a078c8c61cb69')]['starbucks'][0]['coords']['coordinates']
#for star in selected[ObjectId('5dd6b0add27a078c8c61cb69')]['starbucks']:
    #print(star['coords']['coordinates'])

In [101]:
#map_city=folium.Map(location=[37.4440981,-122.1612868], zoom_start=12)

map_city=folium.Map([-122.1615958214, 37.4438754674][::-1], zoom_start=16)
folium.Marker(final_comp[0]['coords']['coordinates'][::-1],
                    radius=1,
                    icon=folium.Icon(icon='briefcase',color='orange'), 
#                     tooltip = tooltip
                   ).add_to(map_city)
for star in selected[ObjectId('5dd6b0add27a078c8c61cb69')]['starbucks']:
    folium.Marker(star['coords']['coordinates'][::-1],
                    radius=1,
                    icon=folium.Icon(icon='star',color='red'), 
#                     tooltip = tooltip
                   ).add_to(map_city)

for party in selected[ObjectId('5dd6b0add27a078c8c61cb69')]['party']:
    folium.Marker(party['coords']['coordinates'][::-1],
                    radius=1,
                    icon=folium.Icon(icon='glass',color='blue'), 
#                     tooltip = tooltip
                   ).add_to(map_city)

for school in selected[ObjectId('5dd6b0add27a078c8c61cb69')]['schools']:
    folium.Marker(school['coords']['coordinates'][::-1],
                    radius=1,
                    icon=folium.Icon(icon='education',color='darkpurple'), 
#                     tooltip = tooltip
                   ).add_to(map_city)

for vegan in selected[ObjectId('5dd6b0add27a078c8c61cb69')]['vegan']:
    folium.Marker(vegan['coords']['coordinates'][::-1],
                    radius=1,
                    icon=folium.Icon(icon='cutlery',color='green'), 
#                     tooltip = tooltip
                   ).add_to(map_city)

In [102]:
map_city